## Deliverable 2. Create a Customer Travel Destinations Map.

In [19]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import matplotlib as plt

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv", index_col=0)

city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Flin Flon,CA,54.7682,-101.8650,17.44,27,0,8.05,clear sky
1,Mataura,NZ,-46.1927,168.8643,59.49,90,100,2.17,overcast clouds
2,Kapaa,US,22.0752,-159.3190,79.48,60,0,3.44,clear sky
3,Nizhniy Tsasuchey,RU,50.5167,115.1333,11.07,68,99,23.94,overcast clouds
4,Tiksi,RU,71.6872,128.8694,-16.01,99,100,2.44,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,Kapaa,US,22.0752,-159.3190,79.48,60,0,3.44,clear sky
11,Guerrero Negro,MX,27.9769,-114.0611,72.99,31,71,8.81,broken clouds
16,Bengkulu,ID,-3.8004,102.2655,73.98,96,100,9.26,moderate rain
18,Pisco,PE,-13.7000,-76.2167,80.65,54,0,9.22,clear sky
19,Chuy,UY,-33.6971,-53.4616,71.94,60,12,11.16,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# NA

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Kapaa,US,79.48,clear sky,22.0752,-159.3190,
11,Guerrero Negro,MX,72.99,broken clouds,27.9769,-114.0611,
16,Bengkulu,ID,73.98,moderate rain,-3.8004,102.2655,
18,Pisco,PE,80.65,clear sky,-13.7000,-76.2167,
19,Chuy,UY,71.94,few clouds,-33.6971,-53.4616,
22,Ipameri,BR,72.00,moderate rain,-17.7219,-48.1597,
25,Rikitea,PF,80.11,clear sky,-23.1203,-134.9692,
27,Hilo,US,81.25,few clouds,19.7297,-155.0900,
29,Marilia,BR,77.76,broken clouds,-22.2139,-49.9458,
30,Avarua,CK,84.25,few clouds,-21.2078,-159.7750,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat}, {lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Kapaa,US,79.48,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
11,Guerrero Negro,MX,72.99,broken clouds,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"
16,Bengkulu,ID,73.98,moderate rain,-3.8004,102.2655,Grage Hotel Bengkulu
18,Pisco,PE,80.65,clear sky,-13.7000,-76.2167,Hotel San Isidro Oficial
19,Chuy,UY,71.94,few clouds,-33.6971,-53.4616,Nuevo Hotel Plaza


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
max_temps = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temps,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

# Save figure
fig.savefig()

Figure(layout=FigureLayout(height='420px'))